# 第7章 出力チェック

 - [一、環境設定](#一、環境設定)
 - [二、出力の有害なコンテンツの可能性をチェック](#二、出力の有害なコンテンツの可能性をチェック)
 - [三、出力結果が提供された製品情報と一致するかチェック](#三、出力結果が提供された製品情報と一致するかチェック)

本章では、システムが生成した出力をどのようにチェックするかに焦点を当てます。ユーザーに出力を表示する前に、出力の品質、関連性、安全性をチェックすることは、自動化プロセスやその他のシナリオにおいて提供される応答が非常に重要であることを確保するために重要です。モデレーションAPIを使用して出力を評価する方法を学び、出力を表示する前にモデルの品質評価を向上させるための追加のプロンプトの使用方法を探ります。

## 一、環境設定

前章と同様に、まずOpenAI APIを使用する環境を設定する必要があります

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key  = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

# 以下は環境変数を使用した設定方法の例です。より安全です。参考までに、以降は触れません。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    OpenAI GPT3.5にアクセスする関数をカプセル化

    パラメータ: 
    messages: これはメッセージのリストで、各メッセージはrole（役割）とcontent（内容）を含む辞書です。役割は'system'、'user'、または'assistant'になります。内容は役割のメッセージです。
    model: 呼び出すモデル、デフォルトはgpt-3.5-turbo（ChatGPT）、内部テスト資格を持つユーザーはgpt-4を選択できます
    temperature: これはモデル出力のランダム性を決定します。デフォルトは0で、出力が非常に確定的になります。温度を上げると出力がよりランダムになります。
    max_tokens: これはモデル出力の最大トークン数を決定します。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # モデル出力のランダム性を決定
        max_tokens=max_tokens, # モデル出力の最大トークン数を決定
    )
    return response.choices[0].message["content"]

## 二、出力の有害なコンテンツの可能性をチェック
主にModeration APIの使用

In [ ]:
final_response_to_customer = f"""
The SmartX ProPhone has a 6.1-inch display, 128GB storage, \
12MP dual camera, and 5G. The FotoSnap DSLR Camera \
has a 24.2MP sensor, 1080p video, 3-inch LCD, and \
interchangeable lenses. We have a variety of TVs, including \
the CineView 4K TV with a 55-inch display, 4K resolution, \
HDR, and smart TV features. We also have the SoundMax \
Home Theater system with 5.1 channel, 1000W output, wireless \
subwoofer, and Bluetooth. Do you have any specific questions \
about these products or any other products we offer?
"""
# Moderation 是 OpenAI 的内容审核函数，用于检测这段内容的危害含量

response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 2.6680607e-06,
    "hate/threatening": 1.2194433e-08,
    "self-harm": 8.294434e-07,
    "sexual": 3.41087e-05,
    "sexual/minors": 1.5462567e-07,
    "violence": 6.3285606e-06,
    "violence/graphic": 2.9102332e-06
  },
  "flagged": false
}


In [ ]:
final_response_to_customer = f"""
SmartX ProPhoneは6.1インチディスプレイ、128GBストレージ、\
12MPデュアルカメラ、5Gを搭載しています。FotoSnap デジタル一眼レフカメラは\
24.2MPセンサー、1080pビデオ、3インチLCD、\
交換可能レンズを備えています。当店では、CineView 4Kテレビなど\
55インチディスプレイ、4K解像度、HDR、\
スマートTV機能を備えたさまざまなテレビを取り扱っています。\
また、5.1チャンネル、1000W出力、ワイヤレス\
サブウーファー、Bluetoothを備えたSoundMax\
ホームシアターシステムもございます。これらの製品や\
当店が提供するその他の製品について具体的な質問はございますか？
"""

response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

ご覧のとおり、この出力はフラグ付けされておらず、すべてのカテゴリーで非常に低いスコアを獲得しており、提供された応答が合理的であることを示しています。

全体的に、出力をチェックすることも非常に重要です。例えば、敏感な対象者向けのチャットボットを作成している場合、より低い閾値を使用して出力をフラグ付けできます。一般的に、モデレーション出力がコンテンツがフラグ付けされていることを示している場合、代替の回答で応答したり、新しい応答を生成したりするなど、適切なアクションを実行できます。

モデルが改善されるにつれて、有害な出力を返す可能性もますます低くなっていることに注意してください。

出力をチェックする別の方法は、モデル自体に生成した結果が満足できるものか、定義した基準に合致するかを尋ねることです。これは、生成された出力を入力の一部としてモデルに提供し、出力の品質を評価するよう要求することで実現できます。このような操作をさまざまな方法で行うことができます。例を見てみましょう。

## 三、出力結果が提供された製品情報と一致するかチェック

In [7]:
# 这是一段电子产品相关的信息
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""

#这是顾客的提问
customer_message = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
#判断相关性
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [ ]:
# これは電子製品に関する情報です
system_message = f"""
あなたは、カスタマーサービス担当者の回答が顧客の質問に十分に答えているかを評価し、\
アシスタントが製品情報から引用したすべての事実が正確であることを検証するアシスタントです。
製品情報、ユーザー、カスタマーサービス担当者のメッセージは\
3つのバッククォート（```）で区切られます。
句読点なしでYまたはNの文字で応答してください：
Y - 出力が質問に十分に答えており、かつ応答が製品情報を正しく使用している場合
N - その他の場合

単一の文字のみを出力してください。
"""

# これは顧客の質問です
customer_message = f"""
SmartX ProPhoneとFotoSnapカメラ、デジタル一眼レフについて教えてください。
また、テレビについても教えてください。
"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
顧客のメッセージ: ```{customer_message}```
製品情報: ```{product_information}```
担当者の回答: ```{final_response_to_customer}```

回答は取得した情報を正しく使用していますか？
回答は質問に十分に答えていますか？

YまたはNを出力
"""
# 関連性を判断
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

In [6]:
another_response = "life is like a box of chocolates"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N


In [ ]:
another_response = "人生はチョコレートの箱のようなものです"
q_a_pair = f"""
顧客のメッセージ: ```{customer_message}```
製品情報: ```{product_information}```
担当者の回答: ```{another_response}```

回答は取得した情報を正しく使用していますか？
回答は質問に十分に答えていますか？

YまたはNを出力
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

ご覧のとおり、モデルは生成された出力の品質についてフィードバックを提供できます。このフィードバックを利用して、出力をユーザーに表示するか、新しい応答を生成するかを決定できます。各ユーザークエリに対して複数のモデル応答を生成し、最良の応答を選択してユーザーに表示することも試せます。したがって、さまざまな試行方法があります。

全体的に、モデレーションAPIを使用して出力をチェックすることは良い習慣です。しかし、ほとんどの場合、特にGPT-4のようなより高度なモデルを使用している場合、これは不要かもしれません。

実際、本番環境でこのような手法を採用している人をあまり見かけません。これにより、追加の呼び出しを待つ必要があり、追加のトークンも必要になるため、システムの遅延とコストが増加します。アプリケーションや製品のエラー率が0.0000001%程度の場合、この方法を試すことができるかもしれません。しかし、全体的に、実際のアプリケーションではこの方法を採用することをお勧めしません。

次の章では、入力評価部分、処理部分、出力チェックで学んだすべての内容を統合して、エンドツーエンドのシステムを構築します。